In [10]:
import pandas as pd

# Load the Excel file
excel_path = "data/서울시버스노선별정류소정보(20240507).xlsx"
df = pd.read_excel(excel_path)

df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표
0,122000006,N6703,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
1,122000004,N6701,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
2,100100573,6702,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
3,100100413,6002,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
4,100100412,6001,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
...,...,...,...,...,...,...,...,...
47056,100100589,N61,183,116000142,17233,개봉역,126.856720,37.493978
47057,100100589,N61,184,116000144,17235,개봉1동사거리.개봉푸르지오아파트,126.846791,37.501413
47058,100100589,N61,185,114000125,15228,신정학마을아파트,126.843963,37.507421
47059,100100589,N61,186,114000257,15362,신정현대아파트.서부트럭터미널,126.842007,37.509011


In [8]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load the shapefile
shp_path = "data/시군구/sig.shp"
gdf_shp = gpd.read_file(shp_path, encoding='euc-kr')

# Load the Excel file
excel_path = "data/서울시버스노선별정류소정보(20240507).xlsx"
df = pd.read_excel(excel_path)

# Convert the DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['X좌표'], df['Y좌표'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry)

# Set the CRS (Coordinate Reference System)
gdf_points = gdf_points.set_crs("EPSG:4326")
gdf_shp = gdf_shp.to_crs("EPSG:4326")

# Perform spatial join to determine the district
gdf_joined = gpd.sjoin(gdf_points, gdf_shp, how='left', op='within')

# Extract necessary columns
result = gdf_joined[['ROUTE_ID', '노선명', '순번', 'NODE_ID', 'ARS_ID', '정류소명', 'X좌표', 'Y좌표', 'SIG_KOR_NM', 'SIG_CD']]

# Save the result to a new Excel file
result_excel_path = "data/서울시버스노선별정류소정보_with_district.xlsx"
result.to_excel(result_excel_path, index=False)

filtered_result = result[result['SIG_CD'].astype(str).str.startswith('11')]


/Users/minseokpark/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [12]:
filtered_result.to_excel("data/서울시버스노선별정류소정보_seoul_filtered.xlsx", index=False)

filtered_result

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,SIG_KOR_NM,SIG_CD
14,100000020,청와대A01(자율주행),1,100000419,1280,경복궁역(효자로입구),126.974305,37.576715,종로구,11110
15,123900001,송파01,1,123900007,24511,송파호반베르디움더퍼스트,127.140918,37.506850,송파구,11710
16,103900012,성동03-2,1,103900269,4884,왕십리KCC스위첸아파트,127.026824,37.560369,성동구,11200
17,112900002,서대문12,1,112900261,13946,동원베네스트주차장.두산위브입구,126.944217,37.599114,서대문구,11410
18,110900005,노원03,1,110000183,11283,석계역1번출구.A,127.065197,37.614930,노원구,11350
...,...,...,...,...,...,...,...,...,...,...
47056,100100589,N61,183,116000142,17233,개봉역,126.856720,37.493978,구로구,11530
47057,100100589,N61,184,116000144,17235,개봉1동사거리.개봉푸르지오아파트,126.846791,37.501413,구로구,11530
47058,100100589,N61,185,114000125,15228,신정학마을아파트,126.843963,37.507421,양천구,11470
47059,100100589,N61,186,114000257,15362,신정현대아파트.서부트럭터미널,126.842007,37.509011,양천구,11470


In [11]:
gdf_shp

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((127.00864 37.58047, 127.00871 37.580..."
1,11140,Jung-gu,중구,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,11170,Yongsan-gu,용산구,"POLYGON ((126.96918 37.55566, 126.96917 37.554..."
3,11200,Seongdong-gu,성동구,"POLYGON ((127.04341 37.57234, 127.04524 37.571..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."
...,...,...,...,...
247,52740,Jangsu-gun,장수군,"POLYGON ((127.61996 35.83894, 127.61996 35.838..."
248,52750,Imsil-gun,임실군,"POLYGON ((127.10477 35.61910, 127.10510 35.619..."
249,52770,Sunchang-gun,순창군,"POLYGON ((126.97726 35.55564, 126.97780 35.555..."
250,52790,Gochang-gun,고창군,"MULTIPOLYGON (((126.42760 35.44847, 126.42764 ..."


In [13]:
import pandas as pd

# Load the Excel file
excel_path = "data/저상버스.xlsx"
df4 = pd.read_excel(excel_path)

df4

,노선번호,인가대수,저상대수,보유율
0,1,14,14,100.0
1,100,29,29,100.0
2,101,32,20,62.5
3,102,20,20,100.0
4,105,20,20,100.0
...,...,...,...,...
323,N72은평,5,3,60.0
324,N72중랑,4,4,100.0
325,N73구산,4,4,100.0
326,N75신림,6,6,100.0


In [7]:
import pandas as pd


excel_path = "results/저상버스비율1.csv"
df = pd.read_csv(excel_path)

excel_path1 = "results/저상버스비율2.csv"
df2 = pd.read_csv(excel_path1)


df

,Unnamed: 0,District,Total_Stops,High_Availability_Stops,전체 비율
0,0,강남구,2815,1654,0.587567
1,1,강동구,1228,641,0.521987
2,2,강북구,1539,551,0.358025
3,3,강서구,2163,1077,0.497920
4,4,관악구,1905,1160,0.608924
5,5,광진구,1023,588,0.574780
6,6,구로구,1587,588,0.370510
7,7,금천구,1160,582,0.501724
8,8,노원구,1916,827,0.431628
9,9,도봉구,1165,481,0.412876


In [8]:
df2

,District,0
0,강남구,0.668917
1,강동구,0.621336
2,강북구,0.469786
3,강서구,0.613500
4,관악구,0.663517
5,광진구,0.620723
6,구로구,0.549464
7,금천구,0.556897
8,노원구,0.480689
9,도봉구,0.496996


In [9]:
variance_A = df['전체 비율'].var(ddof=0)

print(variance_A)

variance_B = df2['0'].var(ddof=0)

print(variance_B)

0.008095415473797376
0.0091671296215633
